## Fixed Data:

In [31]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import numpy as np
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

In [2]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO', frequency = 'w')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

In [3]:
Risk_free_rate.tail()

,Date,RiskFree
1078,2022-04-01,0.17
1079,2022-04-08,0.20
1080,2022-04-15,0.27
1081,2022-04-22,0.41
1082,2022-04-29,0.38


## Get Multiple Attributes for Multiple Coins:

In [4]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    ##print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [5]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolList:
    ##print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    ##print('Merging Dataframes...')
    dfName = f"{i}_Raw"
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='inner'), locals()[frameName])
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate, on='Date', how='left')

    ##print(f"{i} data successfully pulled\n")
##print('\nALL DATA SUCCESSFULLY PULLED!')

In [6]:
BTC_Raw.head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
0,2010-07-12,0.08584,295941.9840,9847.0,13932,NaN
1,2010-07-19,0.05050,177517.6000,3758.0,5337,NaN
2,2010-07-26,0.06110,218970.1800,4323.0,3251,NaN
3,2010-08-02,0.06090,222610.8150,1592.0,3268,NaN
4,2010-08-09,0.06529,243665.5445,1299.0,2811,NaN


In [7]:
LTC_Raw.tail()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
463,2022-03-28,128.926292,9.023978e+09,807733,1566202,NaN
464,2022-04-04,111.343007,7.798857e+09,841901,1576883,NaN
465,2022-04-11,109.018201,7.641529e+09,832321,1652475,NaN
466,2022-04-18,104.648982,7.340578e+09,830602,1562294,NaN
467,2022-04-25,99.467612,6.982212e+09,808648,1503462,NaN


In [41]:
NET = pd.DataFrame({'Date':[]})
NET.Date = BTC_Raw.Date

for i in symbolList:
    dfName = f"{i}_Raw"

    copy1 = f"{i}1"
    locals()[copy1] = locals()[dfName].copy()
    copy2 = f"{i}2"
    locals()[copy2] = locals()[dfName].copy()
    locals()[copy2] = locals()[copy2].shift()

    df_NET = f"{i}_NET"
    locals()[df_NET] = pd.DataFrame({'Date':[],f"{i}":[]})
    locals()[df_NET]['Date'] = locals()[copy1]['Date']
    locals()[df_NET][f"{i}"] = np.log(locals()[copy1]['ActiveAddress'])-np.log(locals()[copy2]['ActiveAddress'])


    NET = NET.merge(locals()[df_NET], left_on='Date', right_on='Date', how='outer')
    
NET['30Percentile'] = NET.quantile(q=0.3, numeric_only= True, axis = 1)
NET['70Percentile'] = NET.quantile(q=0.7, numeric_only= True, axis = 1)

C:\ZacharyYang\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [42]:
NET.tail()

,Date,BTC,ETH,LTC,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,...,WAX,WBTC,WETH,wNXM,WTC,YAM,YFI,ZRX,30Percentile,70Percentile
611,2022-03-28,0.029004,-0.838423,0.033521,-0.622188,-0.656780,-0.689544,-1.625967,-1.167241,-1.073733,...,NaN,-0.639724,-0.574962,-0.715126,-1.098612,-1.098612,-0.600077,-0.652200,-0.921058,-0.636794
612,2022-04-04,-0.070992,0.634692,0.006797,0.382879,0.538997,0.656513,1.252763,0.651300,0.727796,...,NaN,0.571381,0.537376,0.773190,1.098612,0.000000,0.646390,0.430304,0.428622,0.727796
613,2022-04-11,0.012006,0.011861,0.046824,-0.070626,1.011601,0.003724,0.112478,-0.043675,-0.179230,...,NaN,-0.004466,-0.024533,0.268264,-inf,0.000000,-0.151977,-0.014118,-0.161771,0.027857
614,2022-04-18,0.017513,0.089242,-0.056119,-0.147586,-1.098612,0.268920,-0.021506,-0.018019,-0.107420,...,NaN,-0.075206,0.100333,-0.288990,inf,-inf,-0.236659,0.585678,-0.168773,0.088810
615,2022-04-25,0.052567,-0.080082,-0.038385,0.060475,0.342945,-0.064539,3.307842,0.009050,-0.063851,...,NaN,0.027366,0.028766,-0.177100,NaN,inf,0.024098,-0.291501,-0.144668,0.058414


In [71]:
NET.apply(lambda x: x[symbolList] <= x['30Percentile'])

KeyError: "None of [Index(['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER',\n       'BAL',\n       ...\n       'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI',\n       'ZRX'],\n      dtype='object', length=124)] are in the [index]"

In [70]:
short = (NET[symbolList] <= -0.144668)
short = short[symbolList]
short.tail()

,BTC,ETH,LTC,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,...,VERI,WaBi,WAX,WBTC,WETH,wNXM,WTC,YAM,YFI,ZRX
611,False,True,False,True,True,True,True,True,True,True,...,True,True,False,True,True,True,True,True,True,True
612,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
613,False,False,False,False,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,True,False
614,False,False,False,True,True,False,False,False,False,True,...,False,True,False,False,False,True,False,True,True,False
615,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True


In [69]:
short = (NET[symbolList] <= NET['30Percentile'])
short = short[symbolList]
short.tail()

,BTC,ETH,LTC,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,...,VERI,WaBi,WAX,WBTC,WETH,wNXM,WTC,YAM,YFI,ZRX
611,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
612,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
613,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
614,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
615,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
test = (NET[symbolList] == NET['BTC'])


,0,1,2,3,4,5,6,7,8,9,...,WAX,WBTC,WETH,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
612,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
613,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
614,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [62]:
 NET[symbolList].values > NET['70Percentile'].values


ValueError: operands could not be broadcast together with shapes (616,124) (616,) 

In [40]:
short = (NET[symbolList].values <= NET['30Percentile'].values)
long = (NET[symbolList].values >= NET['70Percentile'].values)
NET_array = NET[symbolList].to_numpy()

ValueError: operands could not be broadcast together with shapes (616,124) (616,2) 

In [ ]:
short = np.dot(short, NET_array)
short = pd.DataFrame(short, columns= symbolList)

short['mean'] = short.mean(axis = 0)
short.head()

,BTC,ETH,LTC,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,...,WaBi,WAX,WBTC,WETH,wNXM,WTC,YAM,YFI,ZRX,mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
C5.head()

,Date,mean,30Percentile,70Percentile
0,2010-07-12,NaN,NaN,NaN
1,2010-07-19,NaN,NaN,NaN
2,2010-07-26,NaN,NaN,NaN
3,2010-08-02,NaN,NaN,NaN
4,2010-08-09,NaN,NaN,NaN


In [ ]:
LTC_All.head()

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,NET
0,2013-05-13,3.149100,5.601635e+07,45636,63963,0.02,NaN
1,2013-05-20,3.150000,5.668063e+07,40096,57330,0.01,-0.109481
2,2013-05-27,2.830000,5.150006e+07,34165,49767,NaN,-0.141472
3,2013-06-03,2.348333,4.322178e+07,32639,48009,0.03,-0.035964
4,2013-06-10,2.150000,3.999957e+07,31174,49237,0.04,0.025257
